In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from datetime import datetime

In [3]:
ac_data = "C:\\Users\\Laevskiy\\Desktop\\05-22\\merge104sortedbydate.xlsx"
chem_in = "C:\\Users\\Laevskiy\\Desktop\\05-22\\!chem баки питания.xlsx"
chem_out = "C:\\Users\\Laevskiy\\Desktop\\05-22\\!chem окисленная пульпа.xlsx"
chem_fe2 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\!chem Fe2+.xlsx"

In [4]:
WORK_DIR = "C:\\Users\\Laevskiy\\Desktop\\05-22"

In [6]:
def file_save(df: pd.DataFrame, folder: str, name: str) -> None:
    comp = {
        'method': 'zip',
        'archive_name': 'out.csv'
    }

    filepath = Path(f'./{folder}/{name}.zip')

    df.to_csv(filepath, compression=comp)

## Подготовка данных

### Данные АСУТП

In [7]:
df_ac = pd.read_excel(ac_data, index_col=1, parse_dates=True)
df_ac.drop("Unnamed: 0", axis=1, inplace=True)
df_ac.isna().sum()

320PU108M01           16
320PU222M01           16
320RC104DIC02         16
330AC104FI06          16
330AC104FI07          16
330AC104FI08          16
330AC104FI09          16
330AC104FI10          16
330AC104FI51          16
330AC104FI52          16
330AC104FIC40         16
330AC104FICZ01        16
330AC104FICZ02        16
330AC104FICZ03        16
330AC104FICZ04        16
330AC104FICZ05        16
330AC104LICZ01        16
330AC104LICZ01.1      16
330AC104LICZ01.2      16
330AC104PI64          16
330AC104PI65          16
330AC104PIC63A        16
330AC104PIC63A.1      16
330AC104PIZ6          16
330AC104TIC01         16
330AC104TIC02         16
330AC104TIC03         16
330AC104TIC04         16
330AC104TIC05         16
330AC104TIZ61         16
330PT107PIZ01         16
330TK107LIC01         16
330TK107PI11          16
330TK107TI03          16
330TK107TIC02         16
330TK108LIC01         16
330TK108PI02          16
330TK108TIC02         16
380HX107FIC01         16
380HX107LIC0          16


Уберем пропуски


In [8]:
df_ac.dropna(inplace=True)

Усредним все данные с 10 минутных до часовых. Для дальнейшей стыковки с данными ХА.

In [9]:
df_ac_1h = df_ac.resample('H').mean()

In [10]:
df_ac_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26298 entries, 2019-01-01 00:00:00 to 2021-12-31 17:00:00
Freq: H
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   320PU108M01         24623 non-null  float64
 1   320PU222M01         24623 non-null  float64
 2   320RC104DIC02       24623 non-null  float64
 3   330AC104FI06        24623 non-null  float64
 4   330AC104FI07        24623 non-null  float64
 5   330AC104FI08        24623 non-null  float64
 6   330AC104FI09        24623 non-null  float64
 7   330AC104FI10        24623 non-null  float64
 8   330AC104FI51        24623 non-null  float64
 9   330AC104FI52        24623 non-null  float64
 10  330AC104FIC40       24623 non-null  float64
 11  330AC104FICZ01      24623 non-null  float64
 12  330AC104FICZ02      24623 non-null  float64
 13  330AC104FICZ03      24623 non-null  float64
 14  330AC104FICZ04      24623 non-null  float64
 15  330AC104FI

Некоторые даты пропущены в данных, уберем пропуски еще раз.

In [11]:
df_ac_1h.dropna(inplace=True)

### Данные исходного ХА по бакам питания

In [12]:
df_chem_in = pd.read_excel(chem_in, index_col=0, parse_dates=True, usecols="B:H")
df_chem_in = df_chem_in.apply(pd.to_numeric, errors='coerce')
df_chem_in.fillna(0, inplace=True)

In [13]:
df_chem_in.head()

,320 RC-101 S Fe,320 RC-101 S Sобщ,320 RC-101 S Sso42-,320 RC-101 S As,320 RC-101 S Сорг,320 RC-101 S Собщ
Timestamp,,,,,,
2018-12-01 09:00:00,27.10,25.4,0.0,8.90,0.47,1.34
2018-12-02 09:00:00,25.99,26.9,0.0,7.95,0.50,1.33
2018-12-02 13:00:00,25.78,27.2,0.0,7.95,0.48,1.32
2018-12-02 17:00:00,26.71,26.2,0.0,8.38,0.43,1.22
2018-12-03 20:00:00,0.00,25.9,0.0,8.79,0.64,1.10


In [14]:
df_chem_in_1h = df_chem_in.resample('H', convention='start').first()
df_chem_in_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29075 entries, 2018-12-01 09:00:00 to 2022-03-26 19:00:00
Freq: H
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   320 RC-101 S Fe      1502 non-null   float64
 1   320 RC-101 S Sобщ    1502 non-null   float64
 2   320 RC-101 S Sso42-  1502 non-null   float64
 3   320 RC-101 S As      1502 non-null   float64
 4   320 RC-101 S Сорг    1502 non-null   float64
 5   320 RC-101 S Собщ    1502 non-null   float64
dtypes: float64(6)
memory usage: 1.6 MB


In [15]:
cols = ['Fe', 'Stot', 'SO4', 'As', 'Corg', 'Ctot']
df_chem_in_1h.columns = cols

Приведем даты массивов к одному времени. Возьмем ХА только за те дни, что есть данные с АСУТП

In [16]:
min_date = datetime.fromisoformat('2019-01-01 00:00:00')
max_date = datetime.fromisoformat('2021-12-31 17:00:00')

df_chem_in_1h = df_chem_in_1h[min_date:max_date]

Уберем нулевые значения, а вместо них поставим np.nan

In [17]:
df_chem_in_1h.replace({0: np.NaN}, inplace=True)

### Данные ХА по Fe2+

In [20]:
df_chem_fe2 = pd.read_excel(chem_fe2, index_col=0, parse_dates=True, usecols="B,F")
df_chem_fe2 = df_chem_fe2.apply(pd.to_numeric, errors='coerce')
df_chem_fe2.fillna(0, inplace=True)

In [21]:
df_chem_fe2_1h = df_chem_fe2.resample('H').first()
df_chem_fe2_1h.info()
df_chem_fe2_1h.head(400)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29237 entries, 2018-11-29 15:00:00 to 2022-03-31 19:00:00
Freq: H
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   330-ТК-108 L Fe2+  14781 non-null  float64
dtypes: float64(1)
memory usage: 456.8 KB


,330-ТК-108 L Fe2+
Timestamp,
2018-11-29 15:00:00,0.0
2018-11-29 16:00:00,NaN
2018-11-29 17:00:00,NaN
2018-11-29 18:00:00,NaN
2018-11-29 19:00:00,NaN
...,...
2018-12-16 02:00:00,0.0
2018-12-16 03:00:00,NaN
2018-12-16 04:00:00,0.0


In [22]:
df_chem_fe2_1h = df_chem_fe2_1h[min_date:max_date]
df_chem_fe2_1h.fillna(method='ffill', inplace=True)

In [23]:
cols = ['Fe2+']
df_chem_fe2_1h.columns = cols
df_chem_fe2_1h.tail(400)

,Fe2+
Timestamp,
2021-12-15 02:00:00,1.64
2021-12-15 03:00:00,1.52
2021-12-15 04:00:00,1.61
2021-12-15 05:00:00,1.67
2021-12-15 06:00:00,1.55
...,...
2021-12-31 13:00:00,1.51
2021-12-31 14:00:00,1.51
2021-12-31 15:00:00,1.51


### Данные ОТК по плотностям твердого и питания автоклава¶

In [25]:
path_19 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\Nikitos19.xlsx"
path_20 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\Nikitos20.xlsx"
path_21 = "C:\\Users\\Laevskiy\\Desktop\\05-22\\Nikitos21.xlsx"

ds_19 = pd.read_excel(path_19, sheet_name='AC104', usecols='B:D', index_col=0, parse_dates=True)
ds_20 = pd.read_excel(path_20, sheet_name='AC104', usecols='B:D', index_col=0, parse_dates=True)
ds_21 = pd.read_excel(path_21, sheet_name='AC104', usecols='B:D', index_col=0, parse_dates=True)

In [26]:
ds_full = pd.concat([ds_19, ds_20, ds_21])

In [27]:
assert(ds_19.shape[0] + ds_20.shape[0] + ds_21.shape[0] == ds_full.shape[0])

In [28]:
ds_full = ds_full.apply(pd.to_numeric, errors='coerce')

In [29]:
ds_full_1h = ds_full.resample('H', convention='start').first()

In [30]:
ds_full_1h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33133 entries, 2019-01-01 07:00:00 to 2022-10-12 19:00:00
Freq: H
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   D(S)    1661 non-null   float64
 1   D(SL)   1399 non-null   float64
dtypes: float64(2)
memory usage: 776.6 KB


### Финальные проверки массивов данных¶

In [31]:
print(f'Начальная дата: {min_date}, конечная дата: {max_date}.')

print(f'Данные автоклава от: {df_ac_1h.index.min()} до {df_ac_1h.index.max()}')
print(f'Данные баков питания от: {df_chem_in_1h.index.min()} до {df_chem_in_1h.index.max()}')
print(f'Данные по железу 2+ от: {df_chem_fe2_1h.index.min()} до {df_chem_fe2_1h.index.max()}')
print(f'Данные по плотности в баках питания от: {ds_full_1h.index.min()} до {ds_full_1h.index.max()}')


print(f'Число точек по автоклаву {len(df_ac_1h)}, по бакам питания {len(df_chem_in_1h)}, по железу {len(df_chem_fe2_1h)} и по плотности {len(ds_full_1h)}')

Начальная дата: 2019-01-01 00:00:00, конечная дата: 2021-12-31 17:00:00.
Данные автоклава от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные баков питания от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные по железу 2+ от: 2019-01-01 00:00:00 до 2021-12-31 17:00:00
Данные по плотности в баках питания от: 2019-01-01 07:00:00 до 2022-10-12 19:00:00
Число точек по автоклаву 24623, по бакам питания 26298, по железу 26298 и по плотности 33133


In [32]:
file_save(df_ac_1h, WORK_DIR, 'df_ac_1h')

In [33]:
file_save(df_chem_in_1h, WORK_DIR, 'df_chem_in_1h')

In [34]:
file_save(df_chem_fe2_1h, WORK_DIR, 'df_chem_fe2_1h')

In [35]:
file_save(ds_full_1h, WORK_DIR, 'ds_full_1h')

Загрузим ранее сохраненные данные, если не изменений ранее

In [36]:
df_ac_1h = pd.read_csv(f'{WORK_DIR}\df_ac_1h.zip', index_col=0, parse_dates=True)
df_chem_in_1h = pd.read_csv(f'{WORK_DIR}\df_chem_in_1h.zip', index_col=0, parse_dates=True)
df_chem_fe2_1h = pd.read_csv(f'{WORK_DIR}\df_chem_fe2_1h.zip', index_col=0, parse_dates=True)

### Объединение данных в одну таблицу

In [37]:
df1 = df_ac_1h.merge(df_chem_in_1h, how='inner', left_index=True, right_index=True)
df1 = df1.merge(df_chem_fe2_1h, how='inner', left_index=True, right_index=True)
df1 = df1.merge(ds_full_1h, how='inner', left_index=True, right_index=True)

In [38]:
df1.info()
df1.head(100)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24616 entries, 2019-01-01 07:00:00 to 2021-12-31 17:00:00
Data columns (total 52 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   320PU108M01         24616 non-null  float64
 1   320PU222M01         24616 non-null  float64
 2   320RC104DIC02       24616 non-null  float64
 3   330AC104FI06        24616 non-null  float64
 4   330AC104FI07        24616 non-null  float64
 5   330AC104FI08        24616 non-null  float64
 6   330AC104FI09        24616 non-null  float64
 7   330AC104FI10        24616 non-null  float64
 8   330AC104FI51        24616 non-null  float64
 9   330AC104FI52        24616 non-null  float64
 10  330AC104FIC40       24616 non-null  float64
 11  330AC104FICZ01      24616 non-null  float64
 12  330AC104FICZ02      24616 non-null  float64
 13  330AC104FICZ03      24616 non-null  float64
 14  330AC104FICZ04      24616 non-null  float64
 15  330AC104FICZ05    

,320PU108M01,320PU222M01,320RC104DIC02,330AC104FI06,330AC104FI07,330AC104FI08,330AC104FI09,330AC104FI10,330AC104FI51,330AC104FI52,...,OXYGEN/OxyPurityB,Fe,Stot,SO4,As,Corg,Ctot,Fe2+,D(S),D(SL)
2019-01-01 07:00:00,39.462222,0.0,1599.542706,16.330055,29.556427,16.013472,0.798045,0.0,5314.155549,62.697592,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.12,3900.0,1625.0
2019-01-01 08:00:00,37.424444,0.0,1599.383951,16.726346,28.845586,14.337465,0.650660,0.0,5191.722007,60.557992,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.01,NaN,NaN
2019-01-01 09:00:00,39.504444,0.0,1599.549900,15.932027,29.089164,14.253240,0.628135,0.0,5305.890224,59.910129,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.01,NaN,NaN
2019-01-01 10:00:00,39.457222,0.0,1599.649969,16.308161,29.520459,15.951309,0.714080,0.0,5307.226181,62.492149,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.01,NaN,NaN
2019-01-01 11:00:00,39.466667,0.0,1599.581306,16.162954,29.412535,15.971765,0.735621,0.0,5312.830951,62.280596,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-01-05 06:00:00,0.000000,0.0,1619.702106,0.000000,0.028935,-0.017361,0.002894,0.0,-1.007729,0.014468,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.89,NaN,NaN
2019-01-05 07:00:00,0.000000,0.0,1619.920329,0.000048,0.028935,-0.017361,0.002894,0.0,-1.515648,0.014492,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.89,3900.0,NaN
2019-01-05 08:00:00,0.000000,0.0,1620.435455,0.000024,0.028935,-0.017361,0.002894,0.0,-1.447272,0.014492,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.89,NaN,NaN
2019-01-05 09:00:00,0.000000,0.0,1620.779462,0.000024,0.028935,-0.017361,0.002894,0.0,-1.548905,0.014492,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.89,NaN,NaN


Сдвинем данные по 𝐹𝑒2+ на 2 часа назад.

In [39]:
df1['Fe2+'] = df1['Fe2+'].shift(-2)

In [40]:
df1.head()

,320PU108M01,320PU222M01,320RC104DIC02,330AC104FI06,330AC104FI07,330AC104FI08,330AC104FI09,330AC104FI10,330AC104FI51,330AC104FI52,...,OXYGEN/OxyPurityB,Fe,Stot,SO4,As,Corg,Ctot,Fe2+,D(S),D(SL)
2019-01-01 07:00:00,39.462222,0.0,1599.542706,16.330055,29.556427,16.013472,0.798045,0.0,5314.155549,62.697592,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.01,3900.0,1625.0
2019-01-01 08:00:00,37.424444,0.0,1599.383951,16.726346,28.845586,14.337465,0.650660,0.0,5191.722007,60.557992,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.01,NaN,NaN
2019-01-01 09:00:00,39.504444,0.0,1599.549900,15.932027,29.089164,14.253240,0.628135,0.0,5305.890224,59.910129,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.01,NaN,NaN
2019-01-01 10:00:00,39.457222,0.0,1599.649969,16.308161,29.520459,15.951309,0.714080,0.0,5307.226181,62.492149,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.78,NaN,NaN
2019-01-01 11:00:00,39.466667,0.0,1599.581306,16.162954,29.412535,15.971765,0.735621,0.0,5312.830951,62.280596,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.78,NaN,NaN


In [41]:
file_save(df1, WORK_DIR, 'df_total_ds_1h')